# Creating a Cluster Expansion with an additional Ewald electrostatic term
Fitting cluster expansions with an ewald term was proposed by former student Will Richards. See chapter 4.6 of his [thesis](https://ceder.berkeley.edu/theses/Will_Richards_2017.pdf) for details. 

In [1]:
import numpy as np
import json
from monty.serialization import loadfn
from pymatgen.core.structure import Structure
from smol.cofe import ClusterSubspace, StructureWrangler, ClusterExpansion, RegressionData

In [2]:
# load the prim structure
lno_prim = loadfn('data/lno_prim.json')
    
# load the fitting data
with open('data/lno_fitting_data.json', 'r') as f:
    lno_data = [(Structure.from_dict(x['s']), x['toten']) for x in json.load(f)]

### 1) Create the cluster subspace

In [3]:
subspace = ClusterSubspace.from_cutoffs(lno_prim,
                                        cutoffs={2: 5, 3: 4.1},
                                        basis='sinusoid',
                                        supercell_size='O2-')

### 2) Add the ewald term.
An `EwaldTerm` can be added to a cluster expansion to account for long range electrostatic interactions in ionic materials and therefore reduce the cluster complexity required to train the cluster expansion.

In [4]:
from smol.cofe.extern import EwaldTerm

# eta is the screening parameter used in computing
# real/reciprocal space parts in the Ewald summation
# See pymatgen.analysis.ewald.EwaldSummation
subspace.add_external_term(EwaldTerm(eta=None))

#### 2.1) The Electrostatic "correlation"
The last entry in the correlation vector corresponds to the electrostatic "correlation" from the Ewald summation. It essentially is the normalized electrostatic interaction energy. Since it has units of energy it is not rigorously a correlation like the orbit correlations which are unitless.

In [5]:
structure = lno_data[1][0]
corr = subspace.corr_from_structure(structure)

print(f'The Ewald interaction for a structure with'
      f' composition {structure.composition} is: '
      f'\n{corr[-1]} eV/prim')

The Ewald interaction for a structure with composition Li+1 Ni4+5 Ni3+1 O2-12 is: 
-116.41651881128503 eV/prim


### 3) Creating the cluster expansion
Preparing the training data, fiting and creating the cluster expansion with the Ewald term is
same procedure as done for a regular cluster expansion.

In [6]:
from sklearn.linear_model import LinearRegression

# create and add data to the wrangler
wrangler = StructureWrangler(subspace)
for structure, tot_energy in lno_data:
    wrangler.add_data(structure,
                      properties={'total_energy': tot_energy})

# fit the data with an estimator
estimator = LinearRegression(fit_intercept=False)
estimator.fit(wrangler.feature_matrix,
              wrangler.get_property_vector('total_energy'))

# save regression details
reg_data = RegressionData.from_sklearn(estimator,
                                       wrangler.feature_matrix,
                                       wrangler.get_property_vector('total_energy'))
# create the cluster expansion
expansion = ClusterExpansion(subspace,
                             coefficients=estimator.coef_,
                             regression_data=reg_data)

/home/lbluque/Develop/smol/smol/cofe/wrangling/wrangler.py:631: UserWarning: Unable to match Ni4+6 O2-12 with properties {'total_energy': -188.28833} to supercell_structure. Throwing out.
 Error Message: Supercell could not be found from structure
  warnings.warn(
/home/lbluque/Develop/smol/smol/cofe/wrangling/wrangler.py:631: UserWarning: Unable to match Li+2 Ni4+4 Ni3+2 O2-12 with properties {'total_energy': -200.13866} to supercell_structure. Throwing out.
 Error Message: Mapping could not be found from structure.
  warnings.warn(
/home/lbluque/Develop/smol/smol/cofe/wrangling/wrangler.py:631: UserWarning: Unable to match Li+2 Ni3+2 Ni4+4 O2-12 with properties {'total_energy': -200.42049} to supercell_structure. Throwing out.
 Error Message: Mapping could not be found from structure.
  warnings.warn(
/home/lbluque/Develop/smol/smol/cofe/wrangling/wrangler.py:631: UserWarning: Unable to match Li+3 Ni4+4 Ni2+1 Ni3+1 O2-12 with properties {'total_energy': -206.70884} to supercell_struc

#### 3.1) Check the quality of the fit and the "dielectric" constant
We will check the quality of the fit with the simple methods from before.

It is also useful to look at the value of the fit coefficient obtained for
the Ewald interaction, since its inverse can be intepreted as a dielectric constant.

In [7]:
from sklearn.metrics import mean_squared_error, max_error

train_predictions = np.dot(wrangler.feature_matrix,
                           expansion.coefs)
rmse = mean_squared_error(wrangler.get_property_vector('total_energy'),
                          train_predictions, squared=False)
maxer = max_error(wrangler.get_property_vector('total_energy'),
                  train_predictions)

print(f'RMSE {rmse} eV/prim')
print(f'MAX {maxer} eV/prim')
print(f'Fitted dielectric constant {1/expansion.coefs[-1]}')

RMSE 0.007366465328124704 eV/prim
MAX 0.0160518633152833 eV/prim
Fitted dielectric constant 9.413114998055905


### 4) Constraining the value of the "dielectric" constant
Sometimes we want to constrain the obtained value of the dielectric constant, when it is too large or when it is negative (which would mean higher electrostatic interaction energy is more favorable?)

In [8]:
from theorytoolkit.regression import constrain_dielectric
# to use the constrain_dielectric function the subspace used MUST contain an EwaldTerm
# this is a parametrized decorator to the usage is:
# constrain_dielectric(max_dielectric)(the fit method we want)(fitting data)
# the fit method must return the fitted coefficients

# since the sklearn LinearRegression.fit does not return the coefs, we can do this:
def fit(X, y):
    estimator.fit(X, y)
    return estimator.coef_

coefs = constrain_dielectric(max_dielectric=5)(fit)(wrangler.feature_matrix,
                                                    wrangler.get_property_vector('total_energy'))
train_predictions = np.dot(wrangler.feature_matrix,
                           expansion.coefs)
rmse = mean_squared_error(wrangler.get_property_vector('total_energy'),
                          train_predictions, squared=False)
maxer = max_error(wrangler.get_property_vector('total_energy'),
                  train_predictions)

print(f'RMSE {rmse} eV/prim')
print(f'MAX {maxer} eV/prim')
print(f'Fitted dielectric constant {1/coefs[-1]}')

RMSE 0.007366465328124704 eV/prim
MAX 0.0160518633152833 eV/prim
Fitted dielectric constant 5.0


#### 4.1) If you want to play with decorators you can also do the above in a cleaner looking way.

In [9]:
@constrain_dielectric(max_dielectric=5)
def fit(X, y):
    estimator.fit(X, y)
    return estimator.coef_

coefs = fit(wrangler.feature_matrix,
            wrangler.get_property_vector('total_energy'))

train_predictions = np.dot(wrangler.feature_matrix,
                           expansion.coefs)
rmse = mean_squared_error(wrangler.get_property_vector('total_energy'),
                          train_predictions, squared=False)
maxer = max_error(wrangler.get_property_vector('total_energy'),
                  train_predictions)

print(f'RMSE {rmse} eV/prim')
print(f'MAX {maxer} eV/prim')
print(f'Fitted dielectric constant {1/coefs[-1]}')

RMSE 0.007366465328124704 eV/prim
MAX 0.0160518633152833 eV/prim
Fitted dielectric constant 5.0


### 5) Save work

In [10]:
from smol.io import save_work

file_path = 'data/basic_ce_ewald.mson'
# we can save the subspace as well, but since both the wrangler
# and the expansion have it, there is no need to do so.
save_work(file_path, wrangler, expansion)